In [1]:
import sys
import os

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))

from my_parser import process_zips

In [2]:
ZIP_PATHS = ["../data/datathon_part1.zip", "../data/datathon_part2.zip", "../data/datathon_part3.zip", "../data/datathon_part4.zip"]
all_clients = process_zips(ZIP_PATHS)

In [3]:
all_clients[0].keys()

dict_keys(['passport', 'client_profile', 'account_form', 'client_description', 'label'])

In [4]:
def set_keys_to_none(d):

    for key in d.keys():
        if isinstance(d[key], dict):
            set_keys_to_none(d[key])
        else:
            d[key] = ""
    return d

In [5]:
tmp = all_clients[4]['client_profile'].copy()
tmp = set_keys_to_none(tmp)
tmp

{'name': '',
 'address': {'city': '',
  'street name': '',
  'street number': '',
  'postal code': ''},
 'country_of_domicile': '',
 'birth_date': '',
 'nationality': '',
 'passport_number': '',
 'passport_issue_date': '',
 'passport_expiry_date': '',
 'gender': '',
 'phone_number': '',
 'email_address': '',
 'marital_status': '',
 'secondary_school': {'name': '', 'graduation_year': ''},
 'higher_education': '',
 'employment_history': '',
 'aum': {'savings': '', 'inheritance': '', 'real_estate_value': ''},
 'inheritance_details': {'relationship': '',
  'inheritance year': '',
  'profession': ''},
 'real_estate_details': '',
 'investment_risk_profile': '',
 'investment_horizon': '',
 'investment_experience': '',
 'type_of_mandate': '',
 'preferred_markets': '',
 'currency': ''}

In [6]:
all_clients[4]['client_description']

{'Summary Note': "The RM is Virtanen Laine Viitanen's brother-in-law, having married his's sister. This family connection has strengthened their professional collaboration, allowing for seamless communication and trust.\nWith a career spanning nearly 26 years, Virtanen Laine Viitanen has established himself as a key player in the industry.\nAfter extensive research, he chose Julius Baer for its strong track record in managing high-net-worth clients.\n",
 'Family Background': 'Virtanen Laine Viitanen and Johanna have been married since 2003. Their children are named Salo, Rissanen and Mäkinen.',
 'Education Background': 'Virtanen obtained his secondary school diploma from Riihimäen lukio in 1993.\nVirtanen attended University of Jyväskylä and graduated in 1997.\n',
 'Occupation History': 'Having worked for over 26 years, Virtanen Laine Viitanen brings a wealth of experience and expertise to his current role. He is known for his strategic thinking and ability to drive results.\nVirtanen 

In [7]:
client_id = 4
with open("client_data.txt", "w") as f:
    for key in all_clients[client_id].keys():
        f.write(f"{key}:\n")
        f.write(str(all_clients[client_id][key]))
        f.write("\n\n")
    f.close()

with open("client_data.txt", "r") as f:
    data = f.read()
    print(data)
    f.close()

passport:
{'first_name': 'Virtanen', 'middle_name': 'Laine', 'last_name': 'Viitanen', 'gender': 'M', 'country': 'Finland', 'country_code': 'FIN', 'nationality': 'Finnish', 'birth_date': '1974-05-18', 'passport_number': 'DX5798483', 'passport_mrz': ['P<FINVIITANEN<<VIRTANEN<LAINE<<<<<<<<<<<<<<<<', 'D5X798483FIN740518<<<<<<<<<<<<<<<<<<<<<<<<<<<'], 'passport_issue_date': '2022-04-07', 'passport_expiry_date': '2032-04-06'}

client_profile:
{'name': 'Virtanen Laine Viitanen', 'address': {'city': '', 'street name': '', 'street number': '', 'postal code': ''}, 'country_of_domicile': 'Finland', 'birth_date': '1974-05-18', 'nationality': 'Finnish', 'passport_number': 'DX5798483', 'passport_issue_date': '2022-04-07', 'passport_expiry_date': '2032-04-06', 'gender': 'M', 'phone_number': '+358 045 728 81 32', 'email_address': 'virtanen.viitanen@pp.inet.fi', 'marital_status': 'married', 'secondary_school': {'name': '', 'graduation_year': ''}, 'higher_education': [{'university': 'University of Jyväsk

In [8]:
print(4)

4


In [9]:
import openai
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm  # For the progress bar

# Override the OpenAI API base URL to point to your hosted model
openai.api_base = "http://localhost:8192/v1"

# Define the prompts for parallel requests

base_prompt = "Is there any reason why we should reject this person's application for a bank account? If so, start your answer with YES, otherwise start your answer with NO. Here is the text: "


# Function to send a single request
def send_request(prompt):
    try:
        completion = openai.Completion.create(
            model="davinci-002",  # Replace with the model name your server supports
            prompt=prompt,
            max_tokens=1,
            api_key="sk-no-key-required"
        )
        return completion.choices[0].text.strip()
    except Exception as e:
        return f"Error for prompt '{prompt}': {str(e)}"

for i in range(0, 100, 10):

    prompts = [f"{base_prompt}\n{all_clients[i]['client_description']}" for i in range(i, i+10)]
    # Use ThreadPoolExecutor to send requests in parallel with a progress bar
    results = []
    with ThreadPoolExecutor(max_workers=5) as executor:  # Adjust max_workers as needed
        # Submit all tasks
        futures = {executor.submit(send_request, prompt): prompt for prompt in prompts}
        
        # Use tqdm to show progress
        for future in tqdm(as_completed(futures), total=len(prompts), desc="Processing Requests"):
            results.append(future.result())

    # Print all responses
    for i, result in enumerate(results):
        print(f"Response {i + 1}: {result}")

Processing Requests: 100%|██████████| 10/10 [00:00<00:00, 97541.95it/s]


Response 1: Error for prompt 'Is there any reason why we should reject this person's application for a bank account? If so, start your answer with YES, otherwise start your answer with NO. Here is the text: 
{'Summary Note': "Anna Renate Egger and the RM were introduced at a charity run in Vienna. They ran side by side and discussed their philanthropic goals, which led to a meaningful collaboration.\nHaving worked for over 16 years, Anna Renate Egger brings a wealth of experience and expertise to her current role. She is known for her strategic thinking and ability to drive results.\nJulius Baer's personalized approach and dedication to client success were key factors in her decision to join.\n", 'Family Background': 'They do not have any children.', 'Education Background': 'In 2004, Anna finished her secondary school education at Lycée Français de Vienne.\nAnna completed her university studies at University of Klagenfurt in 2010.\n', 'Occupation History': 'Anna Renate Egger is a seaso

Processing Requests: 100%|██████████| 10/10 [00:00<00:00, 35098.78it/s]

Response 1: Error for prompt 'Is there any reason why we should reject this person's application for a bank account? If so, start your answer with YES, otherwise start your answer with NO. Here is the text: 
{'Summary Note': "The RM is a close family friend of Seppälä Tiainen Voutilainen's parents, having known him since birth. This deep-rooted connection has facilitated a strong and trustworthy professional relationship.\nSeppälä Tiainen Voutilainen is a 40 year old Startup Founder from Germany.\nJulius Baer's exceptional client service and bespoke financial solutions were the main reasons he decided to join.\n", 'Family Background': 'Seppälä Tiainen Voutilainen has been happily married to Aino since 2014. Seppälä and Aino have 2 children, named Tolvanen and Vilma.', 'Education Background': 'In 2004, Seppälä graduated from Lohjan lukio with a secondary school diploma.\nHe then attended University of Helsinki, which he graduated in 2009.\n', 'Occupation History': "Having worked for ove


Processing Requests: 100%|██████████| 10/10 [00:00<00:00, 130663.68it/s]


...

Response 1: Error for prompt 'Is there any reason why we should reject this person's application for a bank account? If so, start your answer with YES, otherwise start your answer with NO. Here is the text: 
{'Summary Note': 'The RM met Ingrid Adèle Michiels during a business conference in Brussels. They attended several sessions together and realized they shared similar professional goals, leading to a fruitful partnership.\nIngrid Adèle Michiels is a 52 year old Director of Operations from Belgium.\nLooking for a private bank with a deep understanding of wealth management, she joined Julius Baer for its expertise.\n', 'Family Background': 'Ingrid Adèle Michiels is currently single. She does not have any children.', 'Education Background': 'Ingrid obtained her secondary school diploma from Institut Notre-Dame de la Paix in 1992.\nIngrid pursued higher education at Vrije Universiteit Brussel, graduating in 1997.\n', 'Occupation History': 'Ingrid Adèle Michiels is a seasoned professiona

Processing Requests: 100%|██████████| 10/10 [00:00<00:00, 13609.03it/s]


Response 1: Error for prompt 'Is there any reason why we should reject this person's application for a bank account? If so, start your answer with YES, otherwise start your answer with NO. Here is the text: 
{'Summary Note': "The RM is a close family friend of Josephine Line Andersen's parents, having known her since birth. This deep-rooted connection has facilitated a strong and trustworthy professional relationship.\nJosephine Line Andersen is a 25 year old Junior Trader from Denmark.\nSeeking a trusted partner for wealth management, she joined Julius Baer for its comprehensive and tailored services.\n", 'Family Background': 'Josephine Line Andersen is currently single. She does not have any children.', 'Education Background': 'Josephine graduated from Holstebro Gymnasium in 2019.\nJosephine pursued higher education at Niels Brock Copenhagen Business College, graduating in 2024.\n', 'Occupation History': 'Josephine Line Andersen is a 25 year old and comes from Denmark.\nIn 2024, Jose

Processing Requests: 100%|██████████| 10/10 [00:00<00:00, 68534.38it/s]

Response 1: Error for prompt 'Is there any reason why we should reject this person's application for a bank account? If so, start your answer with YES, otherwise start your answer with NO. Here is the text: 
{'Summary Note': "Muñoz Gutiérrez Martín and the RM are childhood friends, having attended the same schools and shared many life experiences. Their enduring friendship has laid a solid foundation for their business relationship.\nMuñoz Gutiérrez Martín is a seasoned professional with over 32 years of experience.\nHe was drawn to Julius Baer's holistic financial planning and investment strategies, prompting the decision to join.\n", 'Family Background': 'Muñoz Gutiérrez Martín is currently divorced. He has 1 kid called Encarna.', 'Education Background': 'Muñoz completed his secondary education at IES Rafael Alberti Cádiz in 1988.\nMuñoz completed his university education at University of Valencia in 1994.\n', 'Occupation History': 'Muñoz Gutiérrez Martín is a 55 year old Private Ban

In [10]:
all_clients[234]

{'passport': {'first_name': 'Giorgia',
  'middle_name': 'Alessia',
  'last_name': 'Lotti',
  'gender': 'F',
  'country': 'Italy',
  'country_code': 'ITA',
  'nationality': 'Italian',
  'birth_date': '1965-10-10',
  'passport_number': 'VL7120215',
  'passport_mrz': ['P<ITALOTTI<<GIORGIA<ALESSIA<<<<<<<<<<<<<<<<<<',
   'VL7120215ITA651010<<<<<<<<<<<<<<<<<<<<<<<<<<<'],
  'passport_issue_date': '2020-05-17',
  'passport_expiry_date': '2030-05-16'},
 'client_profile': {'name': 'Giorgia Alessia Lotti',
  'address': {'city': 'Florence',
   'street name': 'Piazza del Popolo',
   'street number': 72,
   'postal code': '38120'},
  'country_of_domicile': 'Italy',
  'birth_date': '1965-10-10',
  'nationality': 'Italian',
  'passport_number': 'VL7120215',
  'passport_issue_date': '2020-05-17',
  'passport_expiry_date': '2030-05-16',
  'gender': 'F',
  'phone_number': '335 1705587',
  'email_address': 'giorgia.lotti@aruba.it',
  'marital_status': 'married',
  'secondary_school': {'name': 'Liceo Music

In [11]:
# Marital status

# 

In [12]:
import datetime as dt
birth_date = all_clients[181]['client_profile']['birth_date']
age = dt.datetime.now().year - dt.datetime.strptime(birth_date, "%Y-%m-%d").year